In [1]:
using BenchmarkTools
using Distributed
using Plots
using Profile
using ProfileVega
using PyCall
using ScikitLearn
using Traceur
using TimerOutputs

In [2]:
@sk_import datasets: (fetch_covtype)

PyObject <function fetch_covtype at 0x7f81619113a0>

In [3]:
digits_data = fetch_covtype();

In [4]:
X_digits = abs.(digits_data["data"]);
X_digits = transpose(X_digits);

In [6]:
function get_gains!(X, current_values, idxs, gains)
    Threads.@threads for i in eachindex(idxs)
        s = 0.0
        for j in eachindex(current_values)
            @inbounds s += @fastmath sqrt(current_values[j] + X[j, idxs[i]])
        end
        gains[i] = s
    end
end;

In [7]:
function calculate_gains!(X, gains, current_values, idxs, current_concave_value_sum)
    get_gains!(X, current_values, idxs, gains)
    
    gains .-= current_concave_value_sum
    return gains
end;

In [9]:
@btime begin
    d, n = size(X_digits);
    idxs = 1:n |> collect

    current_values = zeros(Float64, d)
    current_concave_values = sqrt.(current_values)
    current_concave_values_sum = sum(current_concave_values)

    gains = zeros(Float64, n);
    calculate_gains!(X_digits, gains, current_values, idxs, current_concave_values_sum);
end

  10.594 ms (96 allocations: 8.87 MiB)


581012-element Vector{Float64}:
 221.97327539529405
 219.8966797240838
 271.1471644897934
 274.1822937228027
 215.14557840730572
 216.15354296606503
 226.97207155280068
 224.90252282136385
 231.9978510673305
 228.16724685228837
 238.8754089929368
 274.2626594102293
 266.2365723322574
   ⋮
 169.65537402249007
 168.53603624501983
 166.7954782546634
 164.993267619306
 164.0323204798027
 163.57708363018048
 162.99973719540802
 162.20583123960742
 159.84616192019166
 158.9693746251935
 159.37684265706847
 157.4720088161721